##### Copyright 2018 The TensorFlow Authors.

## Setup

In [1]:
import numpy as np

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import tensorflow as tf

import tensorflow_datasets as tfds
import pandas as pd
import numpy as np
import pandas as pd
import re   
import string
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import sklearn.preprocessing
import keras_nlp
import keras_tuner
import sentencepiece
from tensorflow import keras
from keras.utils import to_categorical
tfds.disable_progress_bar()

/home/birdy/Repos/Jira_Oracle/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using TensorFlow backend


Import `matplotlib` and create a helper function to plot graphs:

In [ ]:
import matplotlib.pyplot as plt


def plot_graphs(history, metric):
  plt.plot(history.history[metric])
  plt.plot(history.history['val_'+metric], '')
  plt.xlabel("Epochs")
  plt.ylabel(metric)
  plt.legend([metric, 'val_'+metric])

In [ ]:
# ## Method for getting input
# from nltk.tokenize import sent_tokenize, word_tokenize
# from nltk.stem import WordNetLemmatizer, PorterStemmer
# from nltk.corpus import stopwords
# import nltk
# nltk.download("punkt")
# nltk.download('wordnet')
# nltk.download("stopwords")

# def parse_text(text):

#     ## Tokenize string into words (and punctuation)
#     word_array = word_tokenize(text)
#     word_array = [word.lower() for word in word_array if word.isalpha()]

#     ## Filter out stop words
#     stop_words = set(stopwords.words("english"))
#     filtered_words = [word for word in word_array if word.casefold() not in stop_words]

#     ## Turn words into lemmatized words
#     lemmatizer = WordNetLemmatizer()
#     lemitized_words = [lemmatizer.lemmatize(word) for word in filtered_words]

#     ## Apply Stemming (Find the roots of similar words)
#     stemmer = PorterStemmer()
#     stemmed_words = [stemmer.stem(word) for word in lemitized_words]

    # joined_words = ""
    # #for word in stemmed_words:
    # joined_words = " ".join(list(stemmed_words))
    # return joined_words

In [ ]:
###makes clean file

# import csv
# csv.field_size_limit(10000000)
# text_list = []
# with open("master_dataset.csv", newline='') as f:
#     #reader = csv.reader(f)
#     data = list(csv.reader(f))
# #print(data[:10])
# for row in data:
    
#     #print(parse_text(row[2]))
#     text_list.append(str(parse_text(row[2])))

# #print(text_list[:10])
# with open("master_dataset_clean.csv", 'w') as f2:
#     #writer = csv.writer(f2)
#     for x in text_list:
#         f2.write(x)
#         f2.write('\n')
        #writer.writerows([row])
            

In [ ]:
tickets = pd.read_csv("master_dataset_clean.csv", skip_blank_lines = False, keep_default_na=False)
tickets = tickets.fillna('[UNK]')
tickets2 = pd.read_csv("labelled_dataset.csv")
tickets3 = pd.DataFrame()

#print(tickets['desc'][:3])
#tickets['desc'] = [parse_text(text) for text in tickets['desc']]
#tickets['desc'] = tickets['desc'].apply(parse_text())
#print(tickets['desc'][:3])

text_column = tickets['desc']
tickets3 = pd.concat([tickets3,text_column], axis = 1)
label_column = tickets2['Grade'].astype(int)
label_column = label_column - 1
tickets3 = pd.concat([tickets3,label_column], axis = 1)

X_train, X_test, y_train, y_test = train_test_split(text_column, label_column, test_size=0.2, shuffle=True)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, shuffle=True)

y_train_cat = to_categorical(y_train, 3)
y_test_cat = to_categorical(y_test, 3)
y_val_cat = to_categorical(y_val, 3)


X_train_dataset = tf.data.Dataset.from_tensor_slices(X_train)
X_test_dataset = tf.data.Dataset.from_tensor_slices(X_test)
X_val_dataset = tf.data.Dataset.from_tensor_slices(X_val)
# y_train_dataset = tf.data.Dataset.from_tensor_slices(y_train)
# y_test_dataset = tf.data.Dataset.from_tensor_slices(y_test)
y_train_dataset = tf.data.Dataset.from_tensor_slices(y_train_cat)
y_test_dataset = tf.data.Dataset.from_tensor_slices(y_test_cat)
y_val_dataset = tf.data.Dataset.from_tensor_slices(y_val_cat)

test_dataset = tf.data.Dataset.zip((X_test_dataset, y_test_dataset))
train_dataset = tf.data.Dataset.zip((X_train_dataset, y_train_dataset))
val_dataset = tf.data.Dataset.zip((X_val_dataset, y_val_dataset))

# print(train_dataset.element_spec)
# print(test_dataset.element_spec)

In [ ]:
np.array(train_dataset)

In [ ]:
for example, label in train_dataset.take(1):
  print('text: ', example.numpy())
  print('label: ', label.numpy())

Next shuffle the data for training and create batches of these `(text, label)` pairs:

In [ ]:
BUFFER_SIZE = 10000 #10000
BATCH_SIZE = 128 #64
VOCAB_SIZE = 2500 #1000

In [ ]:
train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
val_dataset = val_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [ ]:
for example, label in train_dataset.take(1):
  print('texts: ', example.numpy()[:3])
  print()
  print('labels: ', label.numpy()[:3])

## Create the text encoder

The raw text loaded by `tfds` needs to be processed before it can be used in a model. The simplest way to process text for training is using the `TextVectorization` layer. This layer has many capabilities, but this tutorial sticks to the default behavior.

Create the layer, and pass the dataset's text to the layer's `.adapt` method:

In [ ]:
VOCAB_SIZE = 2500 #1000
encoder = tf.keras.layers.TextVectorization(
    max_tokens=VOCAB_SIZE)
encoder.adapt(train_dataset.map(lambda text, label: text))

The `.adapt` method sets the layer's vocabulary. Here are the first 20 tokens. After the padding and unknown tokens they're sorted by frequency:

In [ ]:
vocab = np.array(encoder.get_vocabulary())
vocab[:20]

Once the vocabulary is set, the layer can encode text into indices. The tensors of indices are 0-padded to the longest sequence in the batch (unless you set a fixed `output_sequence_length`):

In [ ]:
encoded_example = encoder(example)[:3].numpy()
encoded_example

In [ ]:
for n in range(3):
  print("Original: ", example[n].numpy())
  print("Round-trip: ", " ".join(vocab[encoded_example[n]]))
  print()

## Create the model

Above is a diagram of the model.

1. This model can be build as a `tf.keras.Sequential`.

2. The first layer is the `encoder`, which converts the text to a sequence of token indices.

3. After the encoder is an embedding layer. An embedding layer stores one vector per word. When called, it converts the sequences of word indices to sequences of vectors. These vectors are trainable. After training (on enough data), words with similar meanings often have similar vectors.

  This index-lookup is much more efficient than the equivalent operation of passing a one-hot encoded vector through a `tf.keras.layers.Dense` layer.

4. A recurrent neural network (RNN) processes sequence input by iterating through the elements. RNNs pass the outputs from one timestep to their input on the next timestep.

  The `tf.keras.layers.Bidirectional` wrapper can also be used with an RNN layer. This propagates the input forward and backwards through the RNN layer and then concatenates the final output.

  * The main advantage of a bidirectional RNN is that the signal from the beginning of the input doesn't need to be processed all the way through every timestep to affect the output.  

  * The main disadvantage of a bidirectional RNN is that you can't efficiently stream predictions as words are being added to the end.

5. After the RNN has converted the sequence to a single vector the two `layers.Dense` do some final processing, and convert from this vector representation to a single logit as the classification output.


The code to implement this is below:

In [ ]:
# model = tf.keras.Sequential([
#     encoder,
#     tf.keras.layers.Embedding(
#         input_dim=len(encoder.get_vocabulary()),
#         output_dim=64,
#         # Use masking to handle the variable sequence lengths
#         mask_zero=True),
#     tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
#     tf.keras.layers.Dense(64, activation='relu'),
#     #tf.keras.layers.Dense(1)
#     tf.keras.layers.Dense(3, activation='softmax')
# ])


model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(len(encoder.get_vocabulary()), 64, mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,  return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    #keras_nlp.layers.FNetEncoder(intermediate_dim=64),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(3, activation='softmax')
])



Please note that Keras sequential model is used here since all the layers in the model only have single input and produce single output. In case you want to use stateful RNN layer, you might want to build your model with Keras functional API or model subclassing so that you can retrieve and reuse the RNN layer states. Please check [Keras RNN guide](https://www.tensorflow.org/guide/keras/rnn#rnn_state_reuse) for more details.

The embedding layer [uses masking](https://www.tensorflow.org/guide/keras/masking_and_padding) to handle the varying sequence-lengths. All the layers after the `Embedding` support masking:

In [ ]:
print([layer.supports_masking for layer in model.layers])

To confirm that this works as expected, evaluate a sentence twice. First, alone so there's no padding to mask:

In [ ]:
# predict on a sample text without padding.

sample_text = ('The movie was cool. The animation and the graphics '
               'were out of this world. I would recommend this movie.')
predictions = model.predict(np.array([sample_text]))
print(predictions[0])

Now, evaluate it again in a batch with a longer sentence. The result should be identical:

In [ ]:
# predict on a sample text with padding

padding = "the " * 2000
predictions = model.predict(np.array([sample_text, padding]))
print(predictions[0])

Compile the Keras model to configure the training process:

In [ ]:
# model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
#               optimizer=tf.keras.optimizers.Adam(1e-4),
#               metrics=['accuracy'])

model.compile(
    loss=keras.losses.CategoricalCrossentropy(from_logits=False),
    #optimizer=tf.keras.optimizers.SGD(),
    optimizer=tf.keras.optimizers.Adam(1e-4),
    metrics=["accuracy"] #tf.keras.metrics.CategoricalAccuracy()
)

In [ ]:
model.summary()

## Train the model

In [ ]:
#find weights for labels

samp0 = 54046
samp1 = 24077
samp2 = 9863
tot_samples = samp0 + samp1 + samp2

w0 = tot_samples / (3*samp0)
w1 = tot_samples / (3*samp1)
w2 = tot_samples / (3*samp2)

In [ ]:
history = model.fit(train_dataset, 
                    epochs=10,
                    validation_data=test_dataset,
                    validation_steps=30,
                    #class_weight = {0:w0,1:w1,2:w2},
                    #use_multiprocessing = True,
                    #workers = 5
                    )
#class weight has made it worse

In [ ]:
test_loss, test_acc = model.evaluate(test_dataset)

print('Test Loss:', test_loss)
print('Test Accuracy:', test_acc)

In [ ]:
plt.figure(figsize=(16, 8))
plt.subplot(1, 2, 1)
plot_graphs(history, 'accuracy')
plt.ylim(None, 1)
plt.subplot(1, 2, 2)
plot_graphs(history, 'loss')
plt.ylim(0, None)

In [ ]:
sample_text = ('The movie was cool. The animation and the graphics '
               'were out of this world. I would recommend this movie.')
predictions = model.predict(np.array([sample_text]))
predictions

In [ ]:
# predict on a sample text without padding.

sample_text = ('The movie was not good. The animation and the graphics '
               'were terrible. I would not recommend this movie.')
predictions = model.predict(np.array([sample_text]))
print(predictions)

print(np.argsmax(predictions[0]))

# Model 2

In [ ]:
train_dataset1 = train_dataset.shuffle(BUFFER_SIZE).batch(32).prefetch(tf.data.AUTOTUNE)
test_dataset1 = test_dataset.batch(32).prefetch(tf.data.AUTOTUNE)
val_dataset1 = val_dataset.batch(32).prefetch(tf.data.AUTOTUNE)



VOCAB_SIZE = 2500 #1000
encoder = tf.keras.layers.TextVectorization(
    max_tokens=VOCAB_SIZE)
encoder.adapt(train_dataset1.map(lambda text, label: text))


# train_dataset2 = train_dataset.shuffle(512).prefetch(16).cache()
# test_dataset2 = test_dataset.shuffle(512).prefetch(16).cache()

In [ ]:
# model2 = tf.keras.Sequential([
#     encoder,
#     tf.keras.layers.Embedding(len(encoder.get_vocabulary()), 128, mask_zero=True),
#     tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
#     keras_nlp.layers.FNetEncoder(intermediate_dim=128),
#     keras_nlp.layers.FNetEncoder(intermediate_dim=128),
#     keras_nlp.layers.FNetEncoder(intermediate_dim=128),
#     tf.keras.layers.Dropout(0.1),
#     tf.keras.layers.Dense(3, activation='softmax')
# ])

model2 = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(len(encoder.get_vocabulary()), 8, mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(4)),
    keras_nlp.layers.FNetEncoder(intermediate_dim=8),
    keras_nlp.layers.FNetEncoder(intermediate_dim=8),
    keras_nlp.layers.FNetEncoder(intermediate_dim=8),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(3, activation='softmax')
])
# input_ids = keras.Input(shape=(None,), dtype="int64", name="input_ids")
# x = encoder((input_ids))
# # x = keras_nlp.layers.TokenAndPositionEmbedding(
# #     vocabulary_size=VOCAB_SIZE,
# #     sequence_length=512,
# #     embedding_dim=128,
# #     mask_zero=True,
# # )(input_ids)

# x = keras_nlp.layers.FNetEncoder(intermediate_dim=128)(inputs=x)
# x = keras_nlp.layers.FNetEncoder(intermediate_dim=128)(inputs=x)
# x = keras_nlp.layers.FNetEncoder(intermediate_dim=128)(inputs=x)


# x = keras.layers.GlobalAveragePooling1D()(x)
# x = keras.layers.Dropout(0.1)(x)
# outputs = tf.keras.layers.Dense(3, activation='softmax')(x)

# fnet_classifier = keras.Model(input_ids, outputs, name="fnet_classifier")



model2.compile(
    loss=keras.losses.CategoricalCrossentropy(from_logits=False),
    #optimizer=tf.keras.optimizers.SGD(),
    optimizer=tf.keras.optimizers.Adam(1e-4),
    metrics=["accuracy"] #tf.keras.metrics.CategoricalAccuracy()
)

model2.summary()

In [ ]:
epochs = 3
batch_size = 32
eval_batch_size = 32

train_data_size = len(X_test)
steps_per_epoch = int(train_data_size / batch_size)
num_train_steps = steps_per_epoch * epochs
warmup_steps = int(0.1 * num_train_steps)
initial_learning_rate=2e-5

print(steps_per_epoch)


In [ ]:
history2 = model2.fit(train_dataset1, 
                    epochs=100,
                    validation_data=val_dataset1,
                    validation_steps=30,
                    steps_per_epoch=steps_per_epoch
                    #class_weight = {0:w0,1:w1,2:w2},
                    #use_multiprocessing = True,
                    #workers = 5
                    )

In [ ]:
test_loss, test_acc = model2.evaluate(test_dataset1)

print('Test Loss:', test_loss)
print('Test Accuracy:', test_acc)

In [ ]:
model2.evaluate(test_dataset1, batch_size=BATCH_SIZE)


In [ ]:
plt.figure(figsize=(16, 8))
plt.subplot(1, 2, 1)
plot_graphs(history2, 'accuracy')
plt.ylim(None, 1)
plt.subplot(1, 2, 2)
plot_graphs(history2, 'loss')
plt.ylim(0, None)

In [ ]:
# Set up epochs and steps
epochs = 3
batch_size = 32
eval_batch_size = 32

train_data_size = len(X_test)
steps_per_epoch = int(train_data_size / BATCH_SIZE)
num_train_steps = steps_per_epoch * epochs
warmup_steps = int(0.1 * num_train_steps)
initial_learning_rate=2e-5

print(steps_per_epoch)


In [ ]:
#save model
#!mkdir -p saved_model
model2.save('saved_model/FNetEncoder_Model.h5')


In [ ]:
# Recreate the exact same model, including its weights and the optimizer
new_model = tf.keras.models.load_model('saved_model/FNetEncoder_Model.h5')

# Show the model architecture
new_model.summary()



## Use kera_tuner to find optimal hyperparams

In [2]:

# [n.name for n in tf.compat.v1.get_default_graph().as_graph_def().node] 
# tf.compat.v1.reset_default_graph()
#tickets = pd.read_csv("combined_dataset.csv", skip_blank_lines = False, keep_default_na=False)
tickets = pd.read_csv("master_dataset_clean.csv", skip_blank_lines = False, keep_default_na=False)
#tickets = pd.read_csv("combined_dataset2.csv")
tickets = tickets.fillna('[UNK]')
tickets2 = pd.read_csv("labelled_dataset.csv")
tickets2 = pd.read_csv("master_dataset.csv")
tickets3 = pd.DataFrame()

# print(tickets['desc'][:3])
# tickets['desc'] = [parse_text(text) for text in tickets['desc']]
# tickets['desc'] = tickets['desc'].apply(parse_text())
# print(tickets['desc'][:3])

text_column = tickets['desc']
tickets3 = pd.concat([tickets3,text_column], axis = 1)
label_column = tickets2['effort(s)'].astype(int)
#label_column = label_column - 1
tickets3 = pd.concat([tickets3,label_column], axis = 1)

## simplify data ~ 10000
df_low_effort = tickets3[tickets3.Grade == 0].sample(30000)
df_med_effort = tickets3[tickets3.Grade == 1].sample(30000, replace = True)
df_hig_effort = tickets3[tickets3.Grade == 2].sample(30000, replace = True)

#df_effort = tickets3.sample(90000)

# print(len(df_low_effort))
# print(len(df_med_effort))
# print(len(df_hig_effort))

result_df = pd.concat([df_low_effort, df_med_effort, df_hig_effort], axis=0)

text_column = result_df['desc']
label_column = result_df['Grade'].astype(int)
# text_column = df_effort['desc']
# label_column = df_effort['effort(s)'].astype(int)


In [3]:
X_train, X_test, y_train, y_test = train_test_split(text_column, label_column, test_size=0.2, shuffle=True)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, shuffle=True)

# y_train_cat = to_categorical(y_train, 3)
# y_test_cat = to_categorical(y_test, 3)
# y_val_cat = to_categorical(y_val, 3)

# max_train = y_train.max()
# min_train = y_train.min()
# y_train = (y_train - min_train)/(max_train - min_train)
# y_test = (y_test - min_train)/(max_train - min_train)
# y_val = (y_val - min_train)/(max_train - min_train)

# train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train_cat))
# val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val_cat))
# test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test_cat))

train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val))
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))

# test_dataset = tf.data.Dataset.zip((X_test_dataset, y_test_dataset))
# train_dataset = tf.data.Dataset.zip((X_train_dataset, y_train_dataset))
# val_dataset = tf.data.Dataset.zip((X_val_dataset, y_val_dataset))
# train_datasetnoc = tf.data.Dataset.zip((X_train_dataset, y_train_datasetnoc))

In [99]:
train_dataset.take(1).get_single_element()

(<tf.Tensor: shape=(), dtype=string, numpy=b'enabl java portabl flink postcommit test jenkin'>,
 <tf.Tensor: shape=(), dtype=int64, numpy=1800>)

In [7]:
# train_dataset2 = train_dataset.shuffle(10000).batch(32).prefetch(tf.data.AUTOTUNE)
# test_dataset2 = test_dataset.batch(32).prefetch(tf.data.AUTOTUNE)
# val_dataset2 = val_dataset.batch(32).prefetch(tf.data.AUTOTUNE)



# VOCAB_SIZE = 2500 #1000
# encoder = tf.keras.layers.TextVectorization(
#     max_tokens=VOCAB_SIZE)
# encoder.adapt(train_dataset2.map(lambda text, label: text))
# counts = np.bincount(train_targets[:, 0])
samp0 = 54046
samp1 = 24077
samp2 = 9863
tot_samples = samp0 + samp1 + samp2

w0 = tot_samples / (3*samp0)
w1 = tot_samples / (3*samp1)
w2 = tot_samples / (3*samp2)

In [ ]:
len(y_train)

In [4]:
BATCH_SIZE = 64
EPOCHS = 3
MAX_SEQUENCE_LENGTH = 256#512
VOCAB_SIZE = 25993 #25993 15000

EMBED_DIM = 128
INTERMEDIATE_DIM = 512

def train_word_piece(ds, vocab_size, reserved_tokens):
    word_piece_ds = ds.map(lambda x, y: x)
    vocab = keras_nlp.tokenizers.compute_word_piece_vocabulary(
        word_piece_ds.batch(1000).prefetch(tf.data.AUTOTUNE),
        vocabulary_size=vocab_size,
        reserved_tokens=reserved_tokens,
    )
    return vocab


In [15]:
reserved_tokens = ["[PAD]", "[UNK]"]
#train_sentences = [element[0] for element in train_dataset]
vocab = train_word_piece(train_dataset, VOCAB_SIZE, reserved_tokens)


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/birdy/Repos/Jira_Oracle/.venv/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_1150603/2413079574.py", line 3, in <module>
    vocab = train_word_piece(train_dataset, VOCAB_SIZE, reserved_tokens)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_1150603/4070957178.py", line 11, in train_word_piece
    vocab = keras_nlp.tokenizers.compute_word_piece_vocabulary(
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/birdy/Repos/Jira_Oracle/.venv/lib/python3.11/site-packages/keras_nlp/src/tokenizers/word_piece_tokenizer_trainer.py", line 164, in compute_word_piece_vocabulary
    vocab = learner.learn(
            ^^^^^^^^^^^^^^
  File "/home/birdy/Repos/Jira_Oracle/.venv/lib/python3.11/site-packages/tensorflow_text/tools/wordpiece_vocab/wordpiece_tokenizer_lea

In [10]:
with open(r'word_vocab_stem.txt', 'w') as fp:
    for item in vocab:
        # write each item on a new line
        fp.write("%s\n" % item)

In [5]:
with open(r'word_vocab_stem.txt', 'r') as fp:
    vocab = fp.read().splitlines()
    

In [6]:
tokenizer = keras_nlp.tokenizers.WordPieceTokenizer(
    vocabulary=vocab,
    lowercase=False,
    sequence_length=MAX_SEQUENCE_LENGTH,
)


In [7]:
input_sentence_ex = train_dataset.take(1).get_single_element()[0]
input_tokens_ex = tokenizer(input_sentence_ex)

print("Sentence: ", input_sentence_ex)
print("Tokens: ", input_tokens_ex)
print("Recovered text after detokenizing: ", tokenizer.detokenize(input_tokens_ex))


Sentence:  tf.Tensor(b'follow todo remov noformat todo add row noformat updat link ticket noformat fixm http fixm http noformat noformat todo cluster initi flow complet todo add busi lock init method http todo need wait futur init noformat', shape=(), dtype=string)
Tokens:  tf.Tensor(
[  62 1460   84   53 1460   42  171   53   70  152  366   53   82  836
   32   82  836   32   53   53 1460  184  212  596  186 1460   42 1950
  352  236   47   32 1460   44  298  508  236   53    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    

In [103]:
i = 0
len_of_lines = 0
for x in text_column:
    len_of_lines += len(x)
    i += 1
avg_len_lines = len_of_lines/i    
print(avg_len_lines)

221.82580183210965


In [8]:
import datetime
# Clear any logs from previous runs
%rm -rf ./logs/

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
log_dir = "logs/scalers/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=0)

# file_writer = tf.summary.FileWriter(log_dir + "/metrics")
# file_writer.set_as_default()


In [11]:
l = encoder(X_train)
ll = encoder2(X_train)
max(label_column)
#scaler(y_train)


5184000

In [28]:
print(l)
print(ll)
embed= keras_nlp.layers.TokenAndPositionEmbedding(
            vocabulary_size=len(vocab),
            sequence_length=MAX_SEQUENCE_LENGTH,
            embedding_dim=EMBED_DIM,
            mask_zero=True,
        )
outputz = embed(ll)
print(outputz[3][1])
#keras_nlp.layers.PositionEmbedding

tf.Tensor(
[[  59 1174  157 ...    0    0    0]
 [1172 1306 1263 ...    0    0    0]
 [  45  201   61 ...    0    0    0]
 ...
 [ 368  628   80 ...    0    0    0]
 [   1 1532  551 ...    0    0    0]
 [ 366   35   50 ...    0    0    0]], shape=(10000, 256), dtype=int64)
tf.Tensor(
[[  57 1172  155 ...    0    0    0]
 [1170 1304 1261 ...    0    0    0]
 [  43  199   59 ...    0    0    0]
 ...
 [ 366  626   78 ...    0    0    0]
 [1844 5066 1530 ...    0    0    0]
 [ 364   33   48 ...    0    0    0]], shape=(10000, 256), dtype=int32)
tf.Tensor(
[-0.00982499 -0.12772794  0.13453987 -0.02867759 -0.01791551  0.07591633
 -0.01407235 -0.10305414  0.0303663   0.09495379  0.04088686  0.00633177
  0.07807508  0.12348113 -0.04962645  0.06221182 -0.03585226 -0.04159736
  0.04247887 -0.04733267 -0.02745271 -0.07568157 -0.02647848 -0.05995679
 -0.0318312  -0.02928777  0.01596187 -0.08794676 -0.04824246 -0.03926285
  0.02734531 -0.01256751 -0.06733508  0.05182472  0.08768432 -0.01539979
  0.0

In [31]:
%rm -rf ./saved_model/

In [9]:
encoder = tf.keras.layers.TextVectorization(
    max_tokens=VOCAB_SIZE,
    vocabulary = vocab,
    output_sequence_length = MAX_SEQUENCE_LENGTH)

encoder2 = keras_nlp.tokenizers.WordPieceTokenizer(
    vocabulary=vocab,
    lowercase=False,
    sequence_length=MAX_SEQUENCE_LENGTH,
)

def r2_score(y_test, y_pred):
    SS_res =  tf.keras.backend.sum(tf.keras.backend.square( y_test-y_pred )) 
    SS_tot = tf.keras.backend.sum(keras.backend.square( y_test - keras.backend.mean(y_test) ) ) 
    return ( 1 - SS_res/(SS_tot + keras.backend.epsilon()) )
# label_mean = y_train.mean()
# train_std = y_train.std()

# scaler = tf.keras.layers.Rescaling(scale=1./max(label_column), 
#                                     offset=0.0
#                                     )
# scaler = tf.keras.layers.Rescaling(scale=(label - label_mean)/train_std, 
#                                     offset=0.0
#                                     )

In [10]:

#scaler = sklearn.preprocessing.MinMaxScaler()
# scaler.fit_transform(y_train)
# scaler.transform(y_test)
# scaler.transform(y_val)
# scaler = StandardScaler()
# train_features = scaler.fit_transform(train_features)

# val_features = scaler.transform(val_features)
# test_features = scaler.transform(test_features)


#train_dataset.as_numpy_iterator

def format_dataset(sentence, label):
    #label = (label - min(label))/(max(label) - min(label))#(X - min(X)) / (max(X) - min(X))
    sentence = encoder2(sentence)
    #label = scaler.transform((label))
    #return ({"input_ids": sentence}, label)
    return (sentence, label)


def make_dataset(dataset):
    #dataset = dataset.map(format_dataset, num_parallel_calls=tf.data.AUTOTUNE)
    return dataset.cache().shuffle(1000).batch(64).map(format_dataset, num_parallel_calls=tf.data.AUTOTUNE).prefetch(tf.data.AUTOTUNE) #dataset.batch(64).shuffle(256).prefetch(tf.data.AUTOTUNE).cache()

#norm with only train values
# label_mean = y_train.np.mean()
# train_std = y_train.std()

full_train_ds = make_dataset(train_dataset)
train_ds = make_dataset(train_dataset.take(10000))
val_ds = make_dataset(val_dataset)
test_ds = make_dataset(test_dataset)

In [11]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_mean_squared_error', patience=3, restore_best_weights=True)

%export TF_ENABLE_AUTO_MIXED_PRECISION=1
%export TF_XLA_FLAGS=--tf_xla_auto_jit=1
%export TF_GPU_ALLOCATOR=cuda_malloc_async

UsageError: Line magic function `%export` not found.


In [119]:

def hyper_class(hp):
    model = keras.Sequential([
        # keras.Input(shape=(None,), dtype="int64", name="input_ids"),
        # keras.Input(shape=(None,), dtype="int64"),
        # keras_nlp.layers.TokenAndPositionEmbedding(
        #     vocabulary_size=VOCAB_SIZE,
        #     sequence_length=MAX_SEQUENCE_LENGTH,
        #     embedding_dim=EMBED_DIM,
        #     mask_zero=True,
        # ),
        #keras.Input(shape=(None,), dtype="int64"),
        #encoder2,
        keras_nlp.layers.TokenAndPositionEmbedding(
            vocabulary_size=len(vocab),#VOCAB_SIZE,
            sequence_length=MAX_SEQUENCE_LENGTH,
            embedding_dim=64,
            mask_zero=True,
            dtype="float32"
        ),
        # tf.keras.layers.Embedding(len(encoder2.get_vocabulary()),
        #                         #input_length = MAX_SEQUENCE_LENGTH,
        #                         output_dim=hp.Int("output_dim", min_value=8, max_value=128, step=32), 
        #                         mask_zero=True),
        #tf.keras.layers.Flatten(),
        #tf.keras.layers.Reshape((None,MAX_SEQUENCE_LENGTH,128), input_shape=(MAX_SEQUENCE_LENGTH,128)),

        #tf.keras.layers.Reshape((MAX_SEQUENCE_LENGTH,128), input_shape=(None,None,MAX_SEQUENCE_LENGTH,128)),
        #tf.reshape(X, shape=[tf.shape(X)[0]*tf.shape(x)[1],2]),
        #keras.layers.GlobalAveragePooling2D(),
        #tf.keras.layers.Dropout(rate=hp.Float("rate", 0.1, 0.99, sampling="log")),
        # tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(units=hp.Int("units", min_value=32, max_value=512, step=32))),
        # tf.keras.layers.Dropout(rate=hp.Float("rate", 0.1, 0.8, sampling="log")),
        keras_nlp.layers.FNetEncoder(intermediate_dim=hp.Int("intermediate_dim1", min_value=32, max_value=512, step=32),
                                    dropout=hp.Float("dropout1", 0.1, 0.8, sampling="log")),
        keras_nlp.layers.FNetEncoder(intermediate_dim=hp.Int("intermediate_dim2", min_value=32, max_value=512, step=32),
                                    dropout=hp.Float("dropout2", 0.1, 0.8, sampling="log")),
        keras_nlp.layers.FNetEncoder(intermediate_dim=hp.Int("intermediate_dim3", min_value=32, max_value=512, step=32),
                                    dropout=hp.Float("dropout3", 0.1, 0.8, sampling="log")),
        #keras.layers.GlobalAveragePooling2D(),
        keras.layers.GlobalAveragePooling1D(),
        tf.keras.layers.Dropout(rate=hp.Float("rate", 0.1, 0.8, sampling="log")),
        #tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(units=hp.Int("units", min_value=4, max_value=128, step=8), return_sequences=hp.Boolean("return_sequences"))),
        
        # tf.keras.layers.Dense(3, activation='softmax'),
        #tf.keras.layers.Dense(1, activation='sigmoid'),
        tf.keras.layers.Dense(1, activation='sigmoid'),

    ])
    
    # model.compile(
    #     optimizer=tf.keras.optimizers.AdamW(learning_rate=hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log"),
    #                                         weight_decay=hp.Float("weight_decay", min_value=1e-4, max_value=1e-2, sampling="log")), 
    #     loss="categorical_crossentropy", 
    #     metrics=[keras.metrics.CategoricalAccuracy(), 
    #             tf.keras.metrics.AUC(name="roc"),
    #             tf.keras.metrics.AUC(curve='PR', name='pr'),
    #             keras.metrics.CategoricalCrossentropy(),
    #             tf.keras.losses.MeanSquaredError(),
    #             tf.keras.losses.MeanAbsoluteError(),
    #     ],
    #     jit_compile=True
        
    model.compile(
        # optimizer=tf.keras.optimizers.AdamW(learning_rate=hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log"),
        #                                     weight_decay=hp.Float("weight_decay", min_value=1e-4, max_value=1e-2, sampling="log")), 
        optimizer = tf.train.experimental.enable_mixed_precision_graph_rewrite(tf.keras.optimizers.AdamW(
                    learning_rate=hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log"),
                    weight_decay=hp.Float("weight_decay", min_value=1e-4, max_value=1e-2, sampling="log"))),
        loss="mean_squared_error", 
        metrics=[
                #tf.keras.metrics.AUC(name="roc"),
                #tf.keras.metrics.AUC(curve='PR', name='pr'),
                #keras.metrics.CategoricalCrossentropy(),
                r2_score,
                tf.keras.metrics.MeanSquaredError(),
                tf.keras.metrics.MeanAbsoluteError(),
                tf.keras.metrics.MeanAbsolutePercentageError(),
                tf.keras.metrics.MeanSquaredLogarithmicError(),
                tf.keras.metrics.RootMeanSquaredError(),
                #tf.keras.metrics.Accuracy(), 
                'acc'
        ],
        jit_compile=True
    )
    return model



tuner = keras_tuner.Hyperband(
    hypermodel=hyper_class,
    # Objective is one of the keys.
    # Maximize the negative MAE, equivalent to minimize MAE.
    objective=[ #keras_tuner.Objective('val_accuracy', direction="max"),
                keras_tuner.Objective('val_mean_squared_error', direction="min"),
                #eras_tuner.Objective("val_roc", direction="max"),
                #keras_tuner.Objective("val_pr", direction="max"),
                #keras_tuner.Objective("val_categorical_crossentropy", direction="min"),
                ],
    max_epochs=30,
    directory="saved_model",
    project_name="classif_hypertuning",
    hyperband_iterations = 2
)









INFO:tensorflow:Reloading Tuner from saved_model/classif_hypertuning/tuner0.json


INFO:tensorflow:Reloading Tuner from saved_model/classif_hypertuning/tuner0.json


In [14]:

def hyper_class(hp):
    model_unnorm = keras.Sequential([

        keras_nlp.layers.TokenAndPositionEmbedding(
            vocabulary_size=len(vocab),#VOCAB_SIZE,
            sequence_length=MAX_SEQUENCE_LENGTH,
            embedding_dim=64,
            mask_zero=True,
            dtype="float32"
        ),

        keras_nlp.layers.FNetEncoder(intermediate_dim=hp.Int("intermediate_dim1", min_value=32, max_value=512, step=32),
                                    dropout=hp.Float("dropout1", 0.1, 0.8, sampling="log")),
        keras_nlp.layers.FNetEncoder(intermediate_dim=hp.Int("intermediate_dim2", min_value=32, max_value=512, step=32),
                                    dropout=hp.Float("dropout2", 0.1, 0.8, sampling="log")),
        keras_nlp.layers.FNetEncoder(intermediate_dim=hp.Int("intermediate_dim3", min_value=32, max_value=512, step=32),
                                    dropout=hp.Float("dropout3", 0.1, 0.8, sampling="log")),
        #keras.layers.GlobalAveragePooling2D(),
        keras.layers.GlobalAveragePooling1D(),
        tf.keras.layers.Dropout(rate=hp.Float("rate", 0.1, 0.8, sampling="log")),
        #tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(units=hp.Int("units", min_value=4, max_value=128, step=8), return_sequences=hp.Boolean("return_sequences"))),
        
        # tf.keras.layers.Dense(3, activation='softmax'),
        #tf.keras.layers.Dense(1, activation='sigmoid'),
        tf.keras.layers.Dense(1),

    ])
    

        
    model_unnorm.compile(
        optimizer=tf.keras.optimizers.AdamW(learning_rate=hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log"),
                                            weight_decay=hp.Float("weight_decay", min_value=1e-4, max_value=1e-2, sampling="log")), 
        loss="mean_squared_error", 
        metrics=[
                #tf.keras.metrics.AUC(name="roc"),
                #tf.keras.metrics.AUC(curve='PR', name='pr'),
                #keras.metrics.CategoricalCrossentropy(),
                r2_score,
                tf.keras.metrics.MeanSquaredError(),
                tf.keras.metrics.MeanAbsoluteError(),
                tf.keras.metrics.MeanAbsolutePercentageError(),
                tf.keras.metrics.MeanSquaredLogarithmicError(),
                tf.keras.metrics.RootMeanSquaredError(),
                #tf.keras.metrics.Accuracy(), 
                'acc'
        ],
        jit_compile=True
    )
    return model_unnorm


tuner = keras_tuner.Hyperband(
    hypermodel=hyper_class,
    # Objective is one of the keys.
    # Maximize the negative MAE, equivalent to minimize MAE.
    objective=[ #keras_tuner.Objective('val_accuracy', direction="max"),
                keras_tuner.Objective('val_mean_squared_error', direction="min"),
                #eras_tuner.Objective("val_roc", direction="max"),
                #keras_tuner.Objective("val_pr", direction="max"),
                #keras_tuner.Objective("val_categorical_crossentropy", direction="min"),
                ],
    max_epochs=30,
    directory="saved_model",
    project_name="classif_hypertuning_nonorm",
    hyperband_iterations = 2
)

INFO:tensorflow:Reloading Tuner from saved_model/classif_hypertuning_nonorm/tuner0.json


In [15]:
tuner.search(train_ds,
            # x=x_trn, 
            # y=y_trn,
            #class_weight = {0:w0,1:w1,2:w2}, 
            validation_data=val_ds, 
            callbacks = [callback, tensorboard_callback])

# tuner.search(x=train_ds_x,
#             y=train_ds_y, 
#             class_weight = {0:w0,1:w1,2:w2}, 
#             validation_data=(val_ds_x, val_ds_y), 
#             callbacks = [callback, tensorboard_callback])
best_model = tuner.get_best_models()[0]

Trial 85 Complete [00h 03m 10s]
multi_objective: 3706295808.0

Best multi_objective So Far: 1329121408.0
Total elapsed time: 01h 39m 34s

Search: Running Trial #86

Value             |Best Value So Far |Hyperparameter
160               |320               |intermediate_dim1
0.38052           |0.46995           |dropout1
96                |224               |intermediate_dim2
0.63873           |0.4655            |dropout2
480               |448               |intermediate_dim3
0.33553           |0.15393           |dropout3
0.78165           |0.51724           |rate
0.0009476         |0.0067938         |lr
0.0071545         |0.00062834        |weight_decay
2                 |2                 |tuner/epochs
0                 |0                 |tuner/initial_epoch
3                 |3                 |tuner/bracket
0                 |0                 |tuner/round

Epoch 1/2


KeyboardInterrupt: 

Bad pipe message: %s [b' 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/115.0\r\nAccept: text/html,']
Bad pipe message: %s [b'plication/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8\r\nAccept-Language: en-C']
Bad pipe message: %s [b'en-US;q=0.7,en;q=0.3\r\nAccept-Encoding: gzip, deflate, br\r\nConnec']


In [109]:
tuner = keras_tuner.Hyperband(
    hypermodel=hyper_class,
    # Objective is one of the keys.
    # Maximize the negative MAE, equivalent to minimize MAE.
    objective=[ #keras_tuner.Objective('val_accuracy', direction="max"),
                keras_tuner.Objective('val_mean_squared_error', direction="min"),
                #eras_tuner.Objective("val_roc", direction="max"),
                #keras_tuner.Objective("val_pr", direction="max"),
                #keras_tuner.Objective("val_categorical_crossentropy", direction="min"),
                ],
    max_epochs=30,
    directory="saved_model",
    project_name="classif_hypertuning",
    hyperband_iterations = 2
)
#best_trial = tuner.oracle.get_best_trials(1)
#best_trial = tuner.get_best_trials(num_trials=1)
#reloaded_model = tuner.get_best_models()[0] #load_model(best_trial)
best_hyperparams = tuner.get_best_hyperparameters()[0]
top_model = tuner.hypermodel.build(best_hyperparams)
#reloaded_model.build()


INFO:tensorflow:Reloading Tuner from saved_model/classif_hypertuning/tuner0.json


INFO:tensorflow:Reloading Tuner from saved_model/classif_hypertuning/tuner0.json


In [45]:
reloaded_model.save('rnn_models/my_model.keras')
# new_model = tf.keras.models.load_model('my_model.keras')


ValueError: Weights for model 'sequential' have not yet been created. Weights are created when the model is first called on inputs or `build()` is called with an `input_shape`.

In [29]:
%load_ext tensorboard
%tensorboard --logdir=logs #--bind_all
#%reload_ext tensorboard

Launching TensorBoard...

In [17]:
# %killbgscripts tensorboard

All background processes were killed.


In [120]:
# file_writer = tf.summary.FileWriter('/tmp/tb_logs', tf.sess.graph)

#best_model.build(input_shape=(None, 28, 28))
#best_model.summary()
best_hyperparams = tuner.get_best_hyperparameters()[0]
top_model = tuner.hypermodel.build(best_hyperparams)

In [47]:
tuner.results_summary()
#print(best_hyperparams)

Results summary
Results in saved_model/classif_hypertuning
Showing 10 best trials
MultiObjective(name="multi_objective", direction="min"): [Objective(name="val_mean_squared_error", direction="min")]

Trial 0055 summary
Hyperparameters:
intermediate_dim1: 512
dropout1: 0.11428114568757548
intermediate_dim2: 128
dropout2: 0.1880763857651768
intermediate_dim3: 256
dropout3: 0.36738043008209137
rate: 0.24686186951876748
lr: 0.0002502739699524452
weight_decay: 0.007652624915370285
tuner/epochs: 6
tuner/initial_epoch: 0
tuner/bracket: 2
tuner/round: 0
Score: 4.2013736674562097e-05

Trial 0042 summary
Hyperparameters:
intermediate_dim1: 32
dropout1: 0.3009694899001408
intermediate_dim2: 64
dropout2: 0.14883165525899547
intermediate_dim3: 64
dropout3: 0.34158324421371616
rate: 0.24457145401045055
lr: 0.0008444640793833608
weight_decay: 0.00632161088105593
tuner/epochs: 6
tuner/initial_epoch: 2
tuner/bracket: 3
tuner/round: 1
tuner/trial_id: 0024
Score: 4.202299169264734e-05

Trial 0059 summary

In [48]:
#top_model.save('saved_model2/model_hp.keras')
#top_model.save("saved_model2/model_hp3.keras", save_format="h5", include_optimizer=True)
keras.models.save_model(top_model, 'rnn_models/my_model.keras')

ValueError: Weights for model 'sequential_2' have not yet been created. Weights are created when the model is first called on inputs or `build()` is called with an `input_shape`.

In [110]:
top_model.summary()

ValueError: This model has not yet been built. Build the model first by calling `build()` or by calling the model on a batch of data.

In [10]:
best_model = keras.Sequential([
    # keras.Input(shape=(None,), dtype="int64", name="input_ids"),
    # keras.Input(shape=(None,), dtype="int64",),
    # keras_nlp.layers.TokenAndPositionEmbedding(
    #     vocabulary_size=VOCAB_SIZE,
    #     sequence_length=MAX_SEQUENCE_LENGTH,
    #     embedding_dim=EMBED_DIM,
    #     mask_zero=True,
    # ),
    #tf.keras.Input(shape=(1,), dtype=tf.string),

    encoder2,
    keras_nlp.layers.TokenAndPositionEmbedding(
        vocabulary_size=VOCAB_SIZE,
        sequence_length=MAX_SEQUENCE_LENGTH,
        embedding_dim=EMBED_DIM,
        mask_zero=True,
    ),
    # tf.keras.layers.Embedding(len(encoder2.get_vocabulary()),
    #                         #input_length = MAX_SEQUENCE_LENGTH,
    #                         output_dim=hp.Int("output_dim", min_value=8, max_value=128, step=32), 
    #                         mask_zero=True),
    #tf.keras.layers.Flatten(),
    tf.keras.layers.Reshape((600,128), input_shape=(None,None,600,128)),
    #tf.reshape(X, shape=[tf.shape(X)[0]*tf.shape(x)[1],2]),
    #keras.layers.GlobalAveragePooling2D(),
    #tf.keras.layers.Dropout(rate=hp.Float("rate", 0.1, 0.99, sampling="log")),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(units=108)),
    keras_nlp.layers.FNetEncoder(intermediate_dim=104,
                                dropout=0.2888822106618751),
    keras_nlp.layers.FNetEncoder(intermediate_dim=72,
                                dropout=0.1477764407983245),
    keras_nlp.layers.FNetEncoder(intermediate_dim=104,
                                dropout=0.1516567561421836),
    tf.keras.layers.Dropout(rate=0.5964271961021104),
    #tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(units=hp.Int("units", min_value=4, max_value=128, step=8), return_sequences=hp.Boolean("return_sequences"))),
    
    tf.keras.layers.Dense(3, activation='softmax'),
])

In [121]:


top_model.compile(
    optimizer = tf.train.experimental.enable_mixed_precision_graph_rewrite(tf.keras.optimizers.AdamW(
                    learning_rate=0.0002502739699524452,
                    weight_decay=0.007652624915370285)),
    loss="mean_squared_error", 
    metrics=[
            #tf.keras.metrics.AUC(name="roc"),
            #tf.keras.metrics.AUC(curve='PR', name='pr'),
            #keras.metrics.CategoricalCrossentropy(),
            #tf.keras.metrics.R2Score(),
            r2_score,
            tf.keras.metrics.MeanSquaredError(),
            tf.keras.metrics.MeanAbsoluteError(),
            tf.keras.metrics.MeanAbsolutePercentageError(),
            tf.keras.metrics.MeanSquaredLogarithmicError(),
            tf.keras.metrics.RootMeanSquaredError(),
            #tf.keras.metrics.Accuracy(), 
            'acc'
    ],
    jit_compile=True
)

# lr: 0.0002502739699524452
# weight_decay: 0.007652624915370285
# best_model.compile(
#     optimizer=tf.keras.optimizers.AdamW(0.000539411439833416), 
#     loss="categorical_crossentropy", 
#     metrics=[keras.metrics.CategoricalAccuracy(), 
#             tf.keras.metrics.AUC(name="roc"),
#             tf.keras.metrics.AUC(curve='PR', name='pr'),
#             keras.metrics.CategoricalCrossentropy()
#         ])  

In [67]:
checkpoint_filepath = '/tmp/checkpoint'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_mean_squared_error',
    mode='min',
    save_best_only=True)

In [112]:
train_data_size = len(X_train)
steps_per_epoch = int(train_data_size / 64)
#epoches = steps_per_epoch * train_data_size

In [122]:
history2 = top_model.fit(
                    full_train_ds,
                    epochs=3,
                    validation_data=val_ds,
                    validation_steps=30,
                    callbacks = [callback, tensorboard_callback, model_checkpoint_callback],
                    #steps_per_epoch=steps_per_epoch,
                    )

Epoch 1/3


278/990 [=======>......................] - ETA: 7:08 - loss: 1483091328.0000 - r2_score: -0.2534 - mean_squared_error: 1483091328.0000 - mean_absolute_error: 11875.5322 - mean_absolute_percentage_error: 99.9529 - mean_squared_logarithmic_error: 59.7520 - root_mean_squared_error: 38510.9219 - acc: 0.0000e+00

KeyboardInterrupt: 

In [113]:
top_model.summary()

ValueError: This model has not yet been built. Build the model first by calling `build()` or by calling the model on a batch of data.

In [83]:
top_model.evaluate(test_ds)

275/275 [==============================] - 3s 9ms/step - loss: 1.0987e-04 - mean_squared_error: 1.0987e-04 - mean_absolute_error: 0.0021 - mean_absolute_percentage_error: 175.3331 - mean_squared_logarithmic_error: 8.5368e-05 - root_mean_squared_error: 0.0105 - acc: 5.6825e-05


[0.00010986616689478979,
 0.00010986616689478979,
 0.002054939279332757,
 175.33309936523438,
 8.536754467058927e-05,
 0.010481705889105797,
 5.682463961420581e-05]

In [94]:
keras.models.save_model(top_model, 'rnn_models/my_model.tf',
                        overwrite= True,
                        save_format='tf')


INFO:tensorflow:Unsupported signature for serialization: ((IndexedSlicesSpec(TensorShape([None, 64]), tf.float32, tf.int32, tf.int32, TensorShape([None])), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7fcf58ec9610>, 140528528571760), {}).
INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(256, 64), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7fcf58ee7e90>, 140528528571648), {}).
INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(64,), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7fcf58eecd50>, 140528528606432), {}).
INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(64,), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7fcf58ecd4d0>, 140528528605472), {}).
INFO:tensorflow:Unsupported signature for serialization: ((

INFO:tensorflow:Assets written to: rnn_models/my_model.tf/assets


INFO:tensorflow:Assets written to: rnn_models/my_model.tf/assets


In [96]:
json_config = top_model.to_json()
top_model.save_weights("rnn_models/my_model.weights.h5")
top_model.load_weights("rnn_models/my_model.weights.h5")

In [ ]:
print(train_dataset.cardinality().numpy())
for element in train_dataset:
    print(element)
    break

# model3

In [ ]:
# tickets = pd.read_csv("combined_dataset.csv", skip_blank_lines = False, keep_default_na=False)
tickets = pd.read_csv("combined_dataset2.csv", skip_blank_lines = False, keep_default_na=False)
tickets = tickets.fillna('[UNK]')
tickets2 = pd.read_csv("labelled_dataset.csv")
tickets3 = pd.DataFrame()

#print(tickets['desc'][:3])
#tickets['desc'] = [parse_text(text) for text in tickets['desc']]
#tickets['desc'] = tickets['desc'].apply(parse_text())
#print(tickets['desc'][:3])

text_column = tickets['desc']
# tickets3 = pd.concat([tickets3,text_column], axis = 1)
label_column = tickets2['Grade'].astype(int)
label_column = label_column - 1
# tickets3 = pd.concat([tickets3,label_column], axis = 1)

X_train, X_test, y_train, y_test = train_test_split(text_column, label_column, test_size=0.2, shuffle=True)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, shuffle=True)

# y_train_cat = to_categorical(y_train, 3)
# y_test_cat = to_categorical(y_test, 3)
# y_val_cat = to_categorical(y_val, 3)


# X_train_dataset = tf.data.Dataset.from_tensor_slices(X_train)
# X_test_dataset = tf.data.Dataset.from_tensor_slices(X_test)
X_val_dataset = tf.data.Dataset.from_tensor_slices(X_val)
# # y_train_dataset = tf.data.Dataset.from_tensor_slices(y_train)
# # y_test_dataset = tf.data.Dataset.from_tensor_slices(y_test)
# y_train_dataset = tf.data.Dataset.from_tensor_slices(y_train_cat)
# y_test_dataset = tf.data.Dataset.from_tensor_slices(y_test_cat)
y_val_dataset = tf.data.Dataset.from_tensor_slices(y_val)

# test_dataset = tf.data.Dataset.zip((X_test_dataset, y_test_dataset))
# train_dataset = tf.data.Dataset.zip((X_train_dataset, y_train_dataset))
val_dataset = tf.data.Dataset.zip((X_val_dataset, y_val_dataset))

features = list(X_train)
labels = list(y_train)

In [ ]:
text_column[:10]

In [ ]:
samp0 = 54046
samp1 = 24077
samp2 = 9863
tot_samples = samp0 + samp1 + samp2

w0 = tot_samples / (3*samp0)
w1 = tot_samples / (3*samp1)
w2 = tot_samples / (3*samp2)

In [ ]:
import io
preprocessor = keras_nlp.models.DebertaV3Preprocessor.from_preset(
    "deberta_v3_base_en"
)

text_list = list(text_column)
print(text_list[:5])
# Tokenize a batch of single sentences.
preprocessor(text_list)



# # Custom vocabulary.
# bytes_io = io.BytesIO()
# ds = tf.data.Dataset.from_tensor_slices(text_list)
# sentencepiece.SentencePieceTrainer.train(
#     sentence_iterator=ds.as_numpy_iterator(),
#     model_writer=bytes_io,
#     vocab_size=25994,
#     model_type="WORD",
#     pad_id=0,
#     bos_id=1,
#     eos_id=2,
#     unk_id=3,
#     pad_piece="[PAD]",
#     bos_piece="[CLS]",
#     eos_piece="[SEP]",
#     unk_piece="[UNK]",
# )
# tokenizer = keras_nlp.models.DebertaV3Tokenizer(
#     proto=bytes_io.getvalue(),
# )
# preprocessor = keras_nlp.models.DebertaV3Preprocessor(tokenizer)
# preprocessor(text_list)


In [ ]:
import io



bytes_io = io.BytesIO()

# tokenizer = keras_nlp.models.DebertaV3Tokenizer.from_preset(
#     "deberta_v3_base_en",
#     #proto=bytes_io.getvalue(),
# )
preprocessor = keras_nlp.models.DebertaV3Preprocessor.from_preset(
    "deberta_v3_base_en",
    #tokenizer=tokenizer,
    sequence_length=128,
)

# Pretrained classifier.
classifier = keras_nlp.models.DebertaV3Classifier.from_preset(
    "deberta_v3_base_en",
    num_classes=3,
    dropout = 0.2,
    #hidden_dim = 2,
    preprocessor = preprocessor,
    activation = "softmax",
    
)

classifier.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    optimizer=keras.optimizers.Adam(5e-5),
    jit_compile=True,

    metrics=['accuracy', 
        #keras.metrics.SparseCategoricalCrossentropy(),
        'crossentropy',
        #tf.keras.metrics.AUC(name="roc"),
        #tf.keras.metrics.AUC(curve='PR', name='pr'),
    ],
    ) 

# classifier.fit(x=features, y=labels, batch_size=64)
#classifier.predict(x=features, batch_size=64)

classifier.fit(x=features,
                y=labels,
                batch_size=64,
                class_weight = {0:w0,1:w1,2:w2},
                validation_data=val_dataset
                )

# # Re-compile (e.g., with a new learning rate).
# classifier.compile(
#     loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
#     optimizer=keras.optimizers.Adam(5e-5),
#     jit_compile=True,
# )
# # Access backbone programatically (e.g., to change `trainable`).
# classifier.backbone.trainable = False
# # Fit again.
# classifier.fit(x=features, y=labels, batch_size=2)


In [ ]:
classifier.summary()

In [ ]:
print(classifier.get_config())

In [ ]:
results = classifier.evaluate(X_test,y_test)
for name, value in zip(classifier.metrics_names, results):
    print(name, ': ', value)
print()